This notebook is designed to be a brief, but useful overview for how fill in missing values for models as well as using different types of label encoding to transform your data.

We'll use the Iowa Housing dataset, which is an option you have for doing homework III.

In [1]:
# import libraries + data
import pandas as pd
import category_encoders as ce
from sklearn.ensemble import GradientBoostingRegressor

# use your own file path here
df = pd.read_csv(r"C:\Users\Jonat\dat-master\Homework\Unit3\data\iowa_mini.csv")

In [2]:
# declare X & y
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [3]:
# check our null values
X.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotArea          0
Neighborhood     0
OverallQual      0
OverallCond      0
YearBuilt        0
GrLivArea        0
1stFlrSF         0
2ndFlrSF         0
GrLivArea.1      0
FullBath         0
HalfBath         0
GarageType      81
GarageYrBlt     81
GarageFinish    81
GarageCars       0
dtype: int64

When dealing with missing values, you typically have to think through the problem in two steps:

 - 1).  Did the missing data happen randomly or not?
    - If it happened randomly:
      - If so, find a way to impute it
    - If it did not happen randomly:
      - What is the missing data likely encoding for?
      
Missing data that happened randomly means you can basically take an estimated guess as to what it might be, whereas as non-random values likely mean you should fill them in with a very particular value.  Most often the missing value likely means 'Nothing happened', 0, 'Not Available', or so on.

With that said, let's look at the patterns of missing values inside this dataset.

In [4]:
# take a look at value counts
df['GarageType'].value_counts()

Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64

In [5]:
# and for GarageYrBlt
df['GarageYrBlt'].value_counts()

2005.0    65
2006.0    59
2004.0    53
2003.0    50
2007.0    49
          ..
1908.0     1
1927.0     1
1933.0     1
1900.0     1
1906.0     1
Name: GarageYrBlt, Length: 97, dtype: int64

In [6]:
# and Garage Finish
df['GarageFinish'].value_counts()

Unf    605
RFn    422
Fin    352
Name: GarageFinish, dtype: int64

One thing that might stand out to you is that there are no text values for 'No Garage', so this is likely a case where missing values encode the absence of something.  

To double check, we'll look to see if the missing values are correlated or not.

In [8]:
df[['GarageType', 'GarageYrBlt', 'GarageFinish']].isnull().corr()

,GarageType,GarageYrBlt,GarageFinish
GarageType,1.0,1.0,1.0
GarageYrBlt,1.0,1.0,1.0
GarageFinish,1.0,1.0,1.0


Seeing that the correlation of these missing values is 100%, it seems all but a foregone conclusion that these values all mean the same thing.

So in the case of the categorical variables, we'll fill them in with 'None', and for `GarageYrBlt`, we'll fill it in with a value of -1.  (Since tree based models make no assumptions about your data, an odd number that is unlike anything else in your dataset makes it an easy value to split on, thus separating it from the non-null values in your dataset).

But before we do that, let's go ahead and denote the missing values before we impute them.  This is helpful in case we encode our missing values in an incorrect way, the tree would be able to decipher imputed from non-imputed values.

In [10]:
# grab the missing columns
missing_cols = df.loc[:, df.isnull().sum() > 0].columns

# create new columns that mark if something is missing or not
for col in missing_cols:
    # create the new column w/ True/False values
    df[f'{col}_missing'] = pd.isnull(df[col])
    
# now look at our new data
df

,Id,MSSubClass,MSZoning,LotArea,Neighborhood,OverallQual,OverallCond,YearBuilt,GrLivArea,1stFlrSF,...,FullBath,HalfBath,GarageType,GarageYrBlt,GarageFinish,GarageCars,SalePrice,GarageType_missing,GarageYrBlt_missing,GarageFinish_missing
0,1,60,RL,8450,CollgCr,7,5,2003,1710,856,...,2,1,Attchd,2003.0,RFn,2,208500,False,False,False
1,2,20,RL,9600,Veenker,6,8,1976,1262,1262,...,2,0,Attchd,1976.0,RFn,2,181500,False,False,False
2,3,60,RL,11250,CollgCr,7,5,2001,1786,920,...,2,1,Attchd,2001.0,RFn,2,223500,False,False,False
3,4,70,RL,9550,Crawfor,7,5,1915,1717,961,...,1,0,Detchd,1998.0,Unf,3,140000,False,False,False
4,5,60,RL,14260,NoRidge,8,5,2000,2198,1145,...,2,1,Attchd,2000.0,RFn,3,250000,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,7917,Gilbert,6,5,1999,1647,953,...,2,1,Attchd,1999.0,RFn,2,175000,False,False,False
1456,1457,20,RL,13175,NWAmes,6,6,1978,2073,2073,...,2,0,Attchd,1978.0,Unf,2,210000,False,False,False
1457,1458,70,RL,9042,Crawfor,7,9,1941,2340,1188,...,2,0,Attchd,1941.0,RFn,1,266500,False,False,False
1458,1459,20,RL,9717,NAmes,5,6,1950,1078,1078,...,1,0,Attchd,1950.0,Unf,1,142125,False,False,False


Many times these helper columns will not be significant, but in case you are unsure if your imputation is useful or not, it gives your model an easy way to sequester rows that were imputed incorrectly.

With this data added in, let's go ahead and fill in our missing data.

In [13]:
# replace categorical values with None
df[['GarageType', 'GarageFinish']] = df[['GarageType', 'GarageFinish']].fillna('None')

# replace numeric values with -1
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(-1)

And that wraps it up.  

If you were going to impute (guess) what the numbers are, there are typically three different ways to do it:

 - use the average value of a column to fill in the missing value for numeric data
 - use the modal value of a column to fill in the missing value for label based data
 - use a model to predict what the value ought to be based off of the non-null values
 
To do the latter point, you would simply declare `y` to be the column's empty values that you were trying to impute.

Below is a quick example.

In [15]:
df[df['GarageType'].isnull()]

0       False
1       False
2       False
3       False
4       False
        ...  
1455    False
1456    False
1457    False
1458    False
1459    False
Name: GarageType, Length: 1460, dtype: bool

In [24]:
X.isnull().sum()

Id              0
MSSubClass      0
MSZoning        0
LotArea         0
Neighborhood    0
OverallQual     0
OverallCond     0
YearBuilt       0
GrLivArea       0
1stFlrSF        0
2ndFlrSF        0
GrLivArea.1     0
FullBath        0
HalfBath        0
GarageType      0
GarageYrBlt     0
GarageFinish    0
GarageCars      0
dtype: int64

In [35]:
# we'll import a gradient boosting classifier, and reload the data
from sklearn.ensemble import GradientBoostingClassifier

# initialize the model
mod = GradientBoostingClassifier()

# load the data
df = pd.read_csv(r"C:\Users\Jonat\dat-master\Homework\Unit3\data\iowa_mini.csv")

# get the rows where GarageType is not null, declare X & y
idx_vals = df.loc[df['GarageType'].notnull(), :].index
X = df.iloc[idx_vals].drop('SalePrice', axis=1)
y = df.iloc[idx_vals]['GarageType']

X = ce.OrdinalEncoder().fit_transform(X)

# fit the model, and fill in the missing values
mod.fit(X, y)

df['GarageType'] = df['GarageType'].fillna(pd.Series(mod.predict(X)))